# Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

# Mobile Games AB Testing with Cookie Cats

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('gb_sem_9_hw.csv')

Приступим к предварительному изучению данных, которые предстоит проанализировать

In [6]:
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [8]:
df.describe()

,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


Пока все выглядит неплохо: датасет заполнен полностью, типы данных нас устраивают. Отметим только огромный разброс по количеству сыгранных раундов. Проверим, нет ли пересечения между группами:

In [13]:
unique = df.groupby('userid', as_index=False).agg({'version': pd.Series.nunique})

In [14]:
unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   userid   90189 non-null  int64
 1   version  90189 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB


Отлично! Данные между группами не пересекаются. Осталось проверить, поровну ли распределены данные между группами:

In [15]:
df.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

Да, вполне сопоставимо. Можно приступать к анализу данных. Додключим необходимые библиотеки:

In [36]:
from scipy.stats import kstest

Проверим распределение на нормальность в каждой выборке отдельно:

In [31]:
gate_30 = df['sum_gamerounds'][df.version == 'gate_30']
gate_40 = df['sum_gamerounds'][df.version == 'gate_40']

In [35]:
print(kstest (gate_30, 'norm'))
print(kstest (gate_40, 'norm'))

KstestResult(statistic=0.8724176532867202, pvalue=0.0)
KstestResult(statistic=0.8707186187388001, pvalue=0.0)


Тест Колмогорова-Смирнова говорит нам, что мы имеем дело с не нормальным распределением. Будем иметь в виду. Хорошая новость - в обеих тестовых группах данные распределены примерно одинаково. Для тестирования гипотез на данном датасете необходимо использовать критерий Манна-Уитни.

In [37]:
from scipy.stats import mannwhitneyu

Проверим, есть ли разница в однодневном retention rate у выборок:

In [47]:
gate_30_ret1 = df['retention_1'][df.version == 'gate_30']
gate_40_ret1 = df['retention_1'][df.version == 'gate_40']

In [48]:
mannwhitneyu(gate_30_ret1, gate_40_ret1)

MannwhitneyuResult(statistic=1022682813.0, pvalue=0.07441128639919536)

__p-value 0.07 > α = 0.05.__ Принимаем нулевую гипотезу о том, что статистически значимых различий в тестовых группах нет.

Проверим, есть ли разница в семидневном retention rate у выборок:

In [50]:
gate_30_ret7 = df['retention_7'][df.version == 'gate_30']
gate_40_ret7 = df['retention_7'][df.version == 'gate_40']

In [51]:
mannwhitneyu(gate_30_ret7, gate_40_ret7)

MannwhitneyuResult(statistic=1025017239.0, pvalue=0.001554344685704005)

__p-value 0.001 < α = 0.05.__ Отвергаем нулевую гипотезу. Есть статистически значимые отличия в семидневном удержании между группами gate_30 и gate_40.

Осталось выяснить, какая группа лучше:

In [65]:
retention7 = df.retention_7.groupby(df.version).sum()/df.retention_7.groupby(df.version).count()
print(round((retention7*100),2))

version
gate_30    19.02
gate_40    18.20
Name: retention_7, dtype: float64


Как мы видим, семидлневный retention rate группы gate_30 на 0.82% выше.

#### По результатам проведенного  A/B тестирования можно рекомендовать к релизу версию gate_30, т.к. семидневное удержание аудитории выше у нее на 0.82%.
#### Вместе с тем необходимо отметить, что медианное значение сыгранных раундов - 16, и большая часть игроков не встречает события ни на 30-м, ни на 40-м уровне. Возможно, имеет смысл провести тестирование гипотезы с воротами на 15-м уровне.